In [37]:
# -*- coding: utf-8 -*-

import tensorflow as tf
import numpy as np
import random
import math
import os

In [38]:
# ------------------------------------------------------------
# 변수 설정
# ------------------------------------------------------------
STONE_NONE = 0
STONE_PLAYER1 = 1
STONE_PLAYER2 = 2
STONE_MAX = 5

gridSize = 9
nbActions = gridSize * gridSize
nbStates = gridSize * gridSize
hiddenSize = 100
maxMemory = 500
batchSize = 50
epoch = 100
epsilonStart = 1
epsilonDiscount = 0.999
epsilonMinimumValue = 0.1
discount = 0.9
learningRate = 0.0001
winReward = 1

In [39]:
# network architecture

X = tf.placeholder(tf.float32, [None, nbStates])
x_image = tf.reshape(X, [-1, gridSize, gridSize, 1])

def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

W_conv1 = weight_variable([3, 3, 1, 32])
b_conv1 = bias_variable([32])

h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

W_conv2 = weight_variable([3, 3, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

W_fc1 = weight_variable([3 * 3 * 64, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 3*3*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

keep_prob = tf.placeholder("float")
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

W_fc2 = weight_variable([1024, nbActions])
b_fc2 = bias_variable([nbActions])

output_layer=tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

Y = tf.placeholder(tf.float32, [None, nbActions])
cost = tf.reduce_sum(tf.square(Y - output_layer)) / (2 * batchSize)
optimizer = tf.train.AdamOptimizer(learningRate).minimize(cost)

In [42]:
# ------------------------------------------------------------

# ------------------------------------------------------------
# 랜덤값 구함
# ------------------------------------------------------------
def randf(s, e):
    return (float(random.randrange(0, (e - s) * 9999)) / 10000) + s

In [49]:
# ------------------------------------------------------------

# ------------------------------------------------------------
# 오목 환경 클래스
# ------------------------------------------------------------
class OmokEnvironment():

    # --------------------------------
    # 초기화
    # --------------------------------
    def __init__(self, gridSize):
        self.gridSize = gridSize
        self.nbStates = self.gridSize * self.gridSize
        self.state = np.zeros(self.nbStates, dtype=np.uint8)

    # --------------------------------
    # 리셋
    # --------------------------------
    def reset(self):
        self.state = np.zeros(self.nbStates, dtype=np.uint8)

    # --------------------------------
    # 현재 상태 구함
    # --------------------------------
    def getState(self):
        return np.reshape(self.state, (1, self.nbStates))
    
    # grid by grid로 보기 좋게
    def getState2(self):
        return np.reshape(self.state,(self.gridSize,self.gridSize))


    # --------------------------------
    # 플레이어가 바뀐 현재 상태 구함
    # --------------------------------
    def getStateInverse(self):
        tempState = self.state.copy()

        for i in range(self.nbStates):
            if (tempState[i] == STONE_PLAYER1):
                tempState[i] = STONE_PLAYER2
            elif (tempState[i] == STONE_PLAYER2):
                tempState[i] = STONE_PLAYER1

        return np.reshape(tempState, (1, self.nbStates))

    # (64, )인 tempState를 element하나씩 변경?
    # tempState를 (1, 64) 행렬로 리턴

    # --------------------------------
    # 리워드 구함-> 총 8번 검사, 이해하기 어려움...
    # 학습이 느려서 자기 돌과 인접한 위치일 경우 0.05의 보상을 추가로 주었다고 함
    # --------------------------------
    def GetReward(self, player, action):

        # 왼쪽 검사
        if (action % self.gridSize > 0):
            if (self.state[action - 1] == player):
                return 0.05

        # 오른쪽 검사
        if (action % self.gridSize < self.gridSize - 1):
            if (self.state[action + 1] == player):
                return 0.05

        # 위 검사
        if (action - self.gridSize >= 0):
            if (self.state[action - self.gridSize] == player):
                return 0.05

        # 아래 검사
        if (action + self.gridSize < self.nbStates):
            if (self.state[action + self.gridSize] == player):
                return 0.05

        # 왼쪽 위 검사
        if ((action % self.gridSize > 0) and (action - self.gridSize >= 0)):
            if (self.state[action - 1 - self.gridSize] == player):
                return 0.05

        # 오른쪽 위 검사
        if ((action % self.gridSize < self.gridSize - 1) and (action - self.gridSize >= 0)):
            if (self.state[action + 1 - self.gridSize] == player):
                return 0.05

        # 왼쪽 아래 검사
        if ((action % self.gridSize > 0) and (action + self.gridSize < self.nbStates)):
            if (self.state[action - 1 + self.gridSize] == player):
                return 0.05

        # 오른쪽 아래 검사
        if ((action % self.gridSize < self.gridSize - 1) and (action + self.gridSize < self.nbStates)):
            if (self.state[action + 1 + self.gridSize] == player):
                return 0.05

        return 0

    # --------------------------------
    # 매칭 검사
    # --------------------------------
    def CheckMatch(self, player):
        for y in range(self.gridSize):
            for x in range(self.gridSize):

                # --------------------------------
                # 오른쪽 검사
                # --------------------------------
                match = 0

                for i in range(STONE_MAX):
                    if (x + i >= self.gridSize):
                        break

                    if (self.state[y * self.gridSize + x + i] == player):
                        match += 1
                    else:
                        break;

                    if (match >= STONE_MAX):
                        return True

                # --------------------------------
                # 아래쪽 검사
                # --------------------------------
                match = 0

                for i in range(STONE_MAX):
                    if (y + i >= self.gridSize):
                        break

                    if (self.state[(y + i) * self.gridSize + x] == player):
                        match += 1
                    else:
                        break;

                    if (match >= STONE_MAX):
                        return True

                # --------------------------------
                # 오른쪽 대각선 검사
                # --------------------------------
                match = 0

                for i in range(STONE_MAX):
                    if ((x + i >= self.gridSize) or (y + i >= self.gridSize)):
                        break

                    if (self.state[(y + i) * self.gridSize + x + i] == player):
                        match += 1
                    else:
                        break;

                    if (match >= STONE_MAX):
                        return True

                # --------------------------------
                # 왼쪽 대각선 검사
                # --------------------------------
                match = 0

                for i in range(STONE_MAX):
                    if ((x - i < 0) or (y + i >= self.gridSize)):
                        break

                    if (self.state[(y + i) * self.gridSize + x - i] == player):
                        match += 1
                    else:
                        break;

                    if (match >= STONE_MAX):
                        return True

        return False

    # --------------------------------
    # 게임오버 검사
    # 게임오버 여부(True/False)와 리워드를 리턴함 
    # --------------------------------
    def isGameOver(self, player):
        if (self.CheckMatch(STONE_PLAYER1) == True):
            if (player == STONE_PLAYER1):
                return True, winReward
            else:
                return True, 0
        elif (self.CheckMatch(STONE_PLAYER2) == True):
            if (player == STONE_PLAYER1):
                return True, 0
            else:
                return True, winReward
        else:
            for i in range(self.nbStates):
                if (self.state[i] == STONE_NONE):
                    return False, 0
            return True, 0

    # checkMatch를 했을 때, 
    # 1이 이겼다면 1에게 win_reward를 주고 게임을 종료(return True)
    # 2가 이겼다면 2에게 win_reward를 주고 게임을 종료(return True)
    # 승부가 안 났으면 (nbStates, )인 state의 element중에 0인게 있으면 
    # 리워드는 0, return False   

    # --------------------------------
    # 상태 업데이트
    # --------------------------------
    def updateState(self, player, action):
        self.state[action] = player;

    # --------------------------------
    # 행동 수행
    # --------------------------------
    def act(self, player, action):
        self.updateState(player, action)
        gameOver, reward = self.isGameOver(player)

        if (reward == 0):
            reward = self.GetReward(player, action)

        if (player == STONE_PLAYER1):
            nextState = self.getState()
        else:
            nextState = self.getStateInverse()

        return nextState, reward, gameOver

    # action을 받아서 먼저 돌을 둔다.
    # 그다음 게임이 끝났는지 검사를 하고,
    # reward = 0 인 상태라면, 돌을 옆에 두었는지 보고 0.05점을 준다.
    # 내(player1) 차례로 끝났다면 nextSate는 지금 돌을 둔 상태.
    # 상대(player2)의 차례로 끝났다면, 
    # 1->2, 2->1로 바꾸는 Inverse함수를 이용해 1이 게임을 두도록 한다.
    # 이 코드는 multi agent를 지원하는 방식이 아닌 듯.
    # --------------------------------
    # 행동 구함
    # --------------------------------
    def getAction(self, sess, currentState):
        q = sess.run(output_layer, feed_dict={X: currentState, keep_prob: 1.0})

        while (True):
            action = q.argmax()

            if (self.state[action] == STONE_NONE):
                return action
            else:
                q[0, action] = -99999

#     def getAction(self, model, currentState) :
#         q = model.predict(currentState)
#         while(1):
#             action = q.argmax()
            
#             if (self.state[action] == STONE_NONE):
#                  return action
#             else:
#                  q[0, action] = -99999
    

    # 랜덤 행동 구함 -> action을 랜덤으로 선택-> e-greedy 알고리즘임
    # --------------------------------
    # 랜덤 행동 구함
    # --------------------------------
    def getActionRandom(self):
        while (True):
            action = random.randrange(0, nbActions)

            if (self.state[action] == STONE_NONE):
                return action


# 네트워크 결과는 q에 저장된다. 이 중 최대 q를 다음 action으로 한다.
# 지금 구한 action이 만약 이미 있는 자리라면, -9999값을 주고 2번째로 높은 값을 선택.

# ------------------------------------------------------------
# 리플레이 메모리 클래스
# ------------------------------------------------------------
class ReplayMemory:

    # --------------------------------
    # 초기화
    # --------------------------------
    def __init__(self, gridSize, maxMemory, discount):
        self.maxMemory = maxMemory
        self.gridSize = gridSize
        self.nbStates = self.gridSize * self.gridSize
        self.discount = discount

        self.inputState = np.empty((self.maxMemory, self.nbStates), dtype=np.uint8)
        self.actions = np.zeros(self.maxMemory, dtype=np.uint8)
        self.nextState = np.empty((self.maxMemory, self.nbStates), dtype=np.uint8)
        self.gameOver = np.empty(self.maxMemory, dtype=np.bool)
        self.rewards = np.empty(self.maxMemory, dtype=np.int8)
        self.count = 0
        self.current = 0

    # --------------------------------
    # 결과 기억
    # --------------------------------
    def remember(self, currentState, action, reward, nextState, gameOver):
        self.actions[self.current] = action
        self.rewards[self.current] = reward
        self.inputState[self.current, ...] = currentState
        self.nextState[self.current, ...] = nextState
        self.gameOver[self.current] = gameOver
        self.count = max(self.count, self.current + 1)
        self.current = (self.current + 1) % self.maxMemory

    # --------------------------------
    # 배치 구함
    # --------------------------------
    def getBatch(self, model, batchSize, nbActions, nbStates, sess):
        memoryLength = self.count
        chosenBatchSize = min(batchSize, memoryLength)

        inputs = np.zeros((chosenBatchSize, nbStates))
        targets = np.zeros((chosenBatchSize, nbActions))

        for i in range(chosenBatchSize):
            randomIndex = random.randrange(0, memoryLength)
            current_inputState = np.reshape(self.inputState[randomIndex], (1, nbStates))

            target = sess.run(model, feed_dict={X: current_inputState, keep_prob: 1.0})

            current_nextState = np.reshape(self.nextState[randomIndex], (1, nbStates))
            current_outputs = sess.run(model, feed_dict={X: current_nextState, keep_prob: 1.0})

            nextStateMaxQ = np.amax(current_outputs)

            if (nextStateMaxQ > winReward):
                nextStateMaxQ = winReward

            if (self.gameOver[randomIndex] == True):
                target[0, [self.actions[randomIndex]]] = self.rewards[randomIndex]
            else:
                target[0, [self.actions[randomIndex]]] = self.rewards[randomIndex] + self.discount * nextStateMaxQ

            inputs[i] = current_inputState
            targets[i] = target

        return inputs, targets


# ------------------------------------------------------------

# -----------------------------------------------------------
# 게임 플레이 함수
# 환경을 초기화하고, 시작 플레이어는 STONE_PLAYER1 1이다. 
# 게임이 끝날 때 까지 행동 수행과 학습 수행을 반복한다 
# action 취할 때 egreedy 쓴 듯?
# ------------------------------------------------------------
def playGame(env, memory, sess, saver, epsilon, iteration):
    # --------------------------------
    # 게임 반복
    # --------------------------------
    winCount = 0

    for i in range(epoch):
        env.reset()

        err = 0
        gameOver = False
        currentPlayer = STONE_PLAYER1

        cnt = 0;
        while (gameOver != True):
            cnt += 1
            
            # --------------------------------
            # 행동 수행
            # --------------------------------
            action = - 9999

            if (currentPlayer == STONE_PLAYER1):
                currentState = env.getState()
            else:
                currentState = env.getStateInverse()

            if (randf(0, 1) <= epsilon):
                action = env.getActionRandom()
            else:
                action = env.getAction(sess, currentState)

            if (epsilon > epsilonMinimumValue):
                epsilon = epsilon * epsilonDiscount

            nextState, reward, gameOver = env.act(currentPlayer, action)

            if (reward == 1 and currentPlayer == STONE_PLAYER1):
                winCount = winCount + 1

            # --------------------------------
            # 학습 수행
            # --------------------------------
            memory.remember(currentState, action, reward, nextState, gameOver)

            inputs, targets = memory.getBatch(output_layer, batchSize, nbActions, nbStates, sess)

            _, loss = sess.run([optimizer, cost], feed_dict={X: inputs, Y: targets, keep_prob: 1.0})
            err = err + loss

            if (currentPlayer == STONE_PLAYER1):
                currentPlayer = STONE_PLAYER2
            else:
                currentPlayer = STONE_PLAYER1
                
            print(env.getState2())

        #print("Epoch " + str(iteration) + str(i) + ": err = " + str(err) + ": Win count = " + str(winCount) +
        #      " Win ratio = " + str(float(winCount) / float(i + 1) * 100))

        #print(targets)
        print("==================================")
        print("cnt : ",cnt)
        print(env.getState2())
        print("==================================")

        if ((i % 10 == 0) and (i != 0)):
            save_path = saver.save(sess, os.getcwd() + "/OmokModelDeep.ckpt")
            print("Model saved in file: %s" % save_path)


# ------------------------------------------------------------

# ------------------------------------------------------------
# 메인 함수
# ------------------------------------------------------------
def main(_):
    # 환경 인스턴스 생성
    env = OmokEnvironment(gridSize)

    # 리플레이 메모리 인스턴스 생성
    memory = ReplayMemory(gridSize, maxMemory, discount)

    # 텐서플로우 초기화
    sess = tf.Session()
    sess.run(tf.global_variables_initializer())

    # 세이브 설정
    saver = tf.train.Saver()

    # 모델 로드
    if (os.path.isfile(os.getcwd() + "/OmokModelDeep.ckpt.index") == True):
        saver.restore(sess, os.getcwd() + "/OmokModelDeep.ckpt")
        print('Saved model is loaded!')

    # 게임 플레이
    iteration = 0
    while (True):
        playGame(env, memory, sess, saver, epsilonStart, iteration);
        iteration += 1

    # 세션 종료
    sess.close()


# ------------------------------------------------------------
# ------------------------------------------------------------
# 메인 함수 실행
# ------------------------------------------------------------
if __name__ == '__main__':
    tf.app.run()
# ------------------------------------------------------------

[[0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]]
[[0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 2 0 0]
 [0 0 0 0 0 0 0 0 0]]
[[0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 2 0 0]
 [0 0 0 0 0 0 0 0 0]]
[[0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 2 0 0 1 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 2 0 0]
 [0 0 0 0 0 0 0 0 0]]
[[0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 2 0 0 1 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 0 0]
 [0 0 0 0 0 0 2 0 0]
 [0 0 0 0 0 0 0 0 0]]
[[0 2 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 2 0 

[[1 0 0 0 0 0 0 0 1]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 2 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]]
[[1 0 0 0 0 0 0 0 1]
 [0 0 0 0 0 0 0 2 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 2 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]]
[[1 0 0 0 0 0 0 0 1]
 [0 0 0 0 0 0 0 2 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 2 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]]
[[1 0 0 0 0 0 0 0 1]
 [0 0 0 0 0 0 0 2 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 2 0 2 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]]
[[1 0 0 0 0 0 0 0 1]
 [0 0 0 1 0 0 0 2 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 2 0 2 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]]
[[1 0 0 0 0 0 0 0 1]
 [0 0 0 1 0 0 0 2 0]
 [0 0 0 

[[1 0 0 1 2 2 1 2 1]
 [1 2 0 1 0 1 0 2 1]
 [0 1 0 2 0 0 1 2 1]
 [1 1 0 0 0 1 0 1 1]
 [0 0 2 0 2 0 2 2 0]
 [0 2 1 0 2 0 1 1 0]
 [2 1 1 0 0 2 2 2 1]
 [0 0 2 2 2 0 0 0 2]
 [0 0 2 0 1 1 2 0 0]]
[[1 0 0 1 2 2 1 2 1]
 [1 2 0 1 0 1 0 2 1]
 [0 1 0 2 0 0 1 2 1]
 [1 1 0 0 0 1 0 1 1]
 [0 0 2 0 2 0 2 2 0]
 [0 2 1 0 2 0 1 1 2]
 [2 1 1 0 0 2 2 2 1]
 [0 0 2 2 2 0 0 0 2]
 [0 0 2 0 1 1 2 0 0]]
[[1 0 0 1 2 2 1 2 1]
 [1 2 0 1 0 1 0 2 1]
 [0 1 0 2 0 0 1 2 1]
 [1 1 0 0 0 1 0 1 1]
 [0 0 2 0 2 0 2 2 0]
 [1 2 1 0 2 0 1 1 2]
 [2 1 1 0 0 2 2 2 1]
 [0 0 2 2 2 0 0 0 2]
 [0 0 2 0 1 1 2 0 0]]
[[1 0 0 1 2 2 1 2 1]
 [1 2 0 1 0 1 0 2 1]
 [0 1 0 2 0 0 1 2 1]
 [1 1 0 0 0 1 0 1 1]
 [0 0 2 0 2 0 2 2 0]
 [1 2 1 0 2 0 1 1 2]
 [2 1 1 0 0 2 2 2 1]
 [0 0 2 2 2 0 0 0 2]
 [0 0 2 0 1 1 2 2 0]]
[[1 0 0 1 2 2 1 2 1]
 [1 2 0 1 0 1 0 2 1]
 [0 1 0 2 0 0 1 2 1]
 [1 1 0 0 0 1 0 1 1]
 [0 0 2 0 2 0 2 2 0]
 [1 2 1 0 2 0 1 1 2]
 [2 1 1 0 0 2 2 2 1]
 [0 0 2 2 2 1 0 0 2]
 [0 0 2 0 1 1 2 2 0]]
[[1 0 0 1 2 2 1 2 1]
 [1 2 0 1 0 1 0 2 1]
 [0 1 0 

[[0 1 0 1 0 2 2 1 0]
 [0 0 0 1 0 0 0 0 0]
 [2 0 1 0 0 0 0 1 2]
 [0 2 1 2 0 0 0 0 1]
 [1 1 0 0 0 2 0 0 0]
 [0 2 0 0 0 1 0 2 0]
 [0 0 0 0 0 1 0 2 0]
 [0 2 2 0 0 1 1 0 0]
 [0 0 0 0 0 0 0 0 2]]
[[0 1 0 1 0 2 2 1 0]
 [2 0 0 1 0 0 0 0 0]
 [2 0 1 0 0 0 0 1 2]
 [0 2 1 2 0 0 0 0 1]
 [1 1 0 0 0 2 0 0 0]
 [0 2 0 0 0 1 0 2 0]
 [0 0 0 0 0 1 0 2 0]
 [0 2 2 0 0 1 1 0 0]
 [0 0 0 0 0 0 0 0 2]]
[[0 1 0 1 0 2 2 1 0]
 [2 0 0 1 0 0 0 0 0]
 [2 0 1 0 0 0 0 1 2]
 [0 2 1 2 0 0 0 0 1]
 [1 1 0 0 0 2 0 0 0]
 [0 2 0 0 0 1 0 2 0]
 [0 0 0 0 0 1 0 2 0]
 [0 2 2 0 0 1 1 1 0]
 [0 0 0 0 0 0 0 0 2]]
[[0 1 0 1 0 2 2 1 0]
 [2 0 0 1 0 0 0 0 0]
 [2 0 1 0 0 0 0 1 2]
 [0 2 1 2 0 0 0 0 1]
 [1 1 0 2 0 2 0 0 0]
 [0 2 0 0 0 1 0 2 0]
 [0 0 0 0 0 1 0 2 0]
 [0 2 2 0 0 1 1 1 0]
 [0 0 0 0 0 0 0 0 2]]
[[0 1 0 1 1 2 2 1 0]
 [2 0 0 1 0 0 0 0 0]
 [2 0 1 0 0 0 0 1 2]
 [0 2 1 2 0 0 0 0 1]
 [1 1 0 2 0 2 0 0 0]
 [0 2 0 0 0 1 0 2 0]
 [0 0 0 0 0 1 0 2 0]
 [0 2 2 0 0 1 1 1 0]
 [0 0 0 0 0 0 0 0 2]]
[[0 1 0 1 1 2 2 1 0]
 [2 0 0 1 2 0 0 0 0]
 [2 0 1 

[[0 0 1 0 0 0 0 2 0]
 [2 0 0 2 0 1 0 1 1]
 [0 1 1 1 0 0 0 2 0]
 [0 0 0 0 0 0 0 0 0]
 [0 2 2 0 2 0 1 0 0]
 [0 2 1 0 0 1 2 0 0]
 [0 0 0 0 0 0 0 1 2]
 [2 0 0 1 0 2 0 0 0]
 [0 0 0 0 0 0 1 0 0]]
[[0 0 1 0 0 0 0 2 0]
 [2 0 0 2 0 1 0 1 1]
 [0 1 1 1 0 0 0 2 2]
 [0 0 0 0 0 0 0 0 0]
 [0 2 2 0 2 0 1 0 0]
 [0 2 1 0 0 1 2 0 0]
 [0 0 0 0 0 0 0 1 2]
 [2 0 0 1 0 2 0 0 0]
 [0 0 0 0 0 0 1 0 0]]
[[0 0 1 0 0 0 0 2 0]
 [2 0 0 2 0 1 0 1 1]
 [0 1 1 1 0 0 0 2 2]
 [0 0 1 0 0 0 0 0 0]
 [0 2 2 0 2 0 1 0 0]
 [0 2 1 0 0 1 2 0 0]
 [0 0 0 0 0 0 0 1 2]
 [2 0 0 1 0 2 0 0 0]
 [0 0 0 0 0 0 1 0 0]]
[[0 0 1 0 0 0 0 2 0]
 [2 0 0 2 0 1 0 1 1]
 [0 1 1 1 0 0 0 2 2]
 [0 0 1 0 0 0 0 0 0]
 [0 2 2 0 2 0 1 0 0]
 [0 2 1 0 0 1 2 0 0]
 [0 0 0 0 0 0 0 1 2]
 [2 0 0 1 0 2 0 2 0]
 [0 0 0 0 0 0 1 0 0]]
[[0 0 1 0 0 0 0 2 0]
 [2 0 0 2 0 1 0 1 1]
 [0 1 1 1 0 0 0 2 2]
 [0 0 1 0 0 0 0 0 0]
 [0 2 2 0 2 0 1 0 0]
 [0 2 1 0 0 1 2 0 0]
 [0 0 0 0 0 0 0 1 2]
 [2 0 0 1 0 2 0 2 0]
 [1 0 0 0 0 0 1 0 0]]
[[0 0 1 0 0 0 0 2 0]
 [2 2 0 2 0 1 0 1 1]
 [0 1 1 

...

[[0 0 0 2 0 0 0 0 1]
 [1 0 0 1 0 0 2 0 0]
 [0 2 2 0 0 0 0 0 0]
 [0 2 0 2 0 0 0 0 0]
 [0 1 1 2 1 0 2 0 0]
 [0 0 0 0 0 0 1 1 0]
 [0 0 0 0 2 0 0 0 0]
 [0 1 0 0 2 1 1 1 0]
 [2 0 0 2 0 1 0 0 0]]
[[0 0 0 2 0 0 0 0 1]
 [1 0 0 1 0 0 2 0 0]
 [0 2 2 0 0 0 0 0 0]
 [0 2 0 2 0 0 0 0 0]
 [0 1 1 2 1 0 2 0 0]
 [0 0 0 0 0 0 1 1 0]
 [0 0 0 0 2 2 0 0 0]
 [0 1 0 0 2 1 1 1 0]
 [2 0 0 2 0 1 0 0 0]]
[[0 0 0 2 0 0 0 1 1]
 [1 0 0 1 0 0 2 0 0]
 [0 2 2 0 0 0 0 0 0]
 [0 2 0 2 0 0 0 0 0]
 [0 1 1 2 1 0 2 0 0]
 [0 0 0 0 0 0 1 1 0]
 [0 0 0 0 2 2 0 0 0]
 [0 1 0 0 2 1 1 1 0]
 [2 0 0 2 0 1 0 0 0]]
[[0 0 0 2 0 0 0 1 1]
 [1 0 0 1 0 0 2 0 0]
 [0 2 2 0 0 0 0 0 0]
 [0 2 0 2 0 0 0 0 0]
 [0 1 1 2 1 0 2 0 0]
 [0 0 0 0 0 0 1 1 0]
 [0 0 0 0 2 2 0 0 0]
 [2 1 0 0 2 1 1 1 0]
 [2 0 0 2 0 1 0 0 0]]
[[0 0 0 2 0 0 0 1 1]
 [1 0 0 1 0 0 2 0 0]
 [0 2 2 0 0 0 0 0 0]
 [0 2 0 2 0 0 0 0 0]
 [0 1 1 2 1 0 2 0 1]
 [0 0 0 0 0 0 1 1 0]
 [0 0 0 0 2 2 0 0 0]
 [2 1 0 0 2 1 1 1 0]
 [2 0 0 2 0 1 0 0 0]]
[[0 0 0 2 0 0 0 1 1]
 [1 0 0 1 0 0 2 0 0]
 [0 2 2 

[[0 0 0 0 0 0 2 0 0]
 [0 2 0 0 0 0 0 0 0]
 [0 0 0 0 0 2 0 0 0]
 [0 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 2 0]
 [0 0 1 0 0 0 0 1 0]
 [1 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]]
[[0 0 0 0 0 0 2 0 0]
 [0 2 0 0 0 0 0 0 1]
 [0 0 0 0 0 2 0 0 0]
 [0 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 2 0]
 [0 0 1 0 0 0 0 1 0]
 [1 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]]
[[0 0 0 0 0 0 2 0 0]
 [0 2 0 0 0 0 0 0 1]
 [0 0 0 0 0 2 0 0 0]
 [0 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 2 0]
 [0 0 1 0 0 0 0 1 2]
 [1 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]]
[[0 0 0 0 0 0 2 1 0]
 [0 2 0 0 0 0 0 0 1]
 [0 0 0 0 0 2 0 0 0]
 [0 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 2 0]
 [0 0 1 0 0 0 0 1 2]
 [1 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]]
[[0 0 0 0 0 0 2 1 0]
 [0 2 0 0 0 0 0 0 1]
 [0 0 0 0 0 2 0 0 0]
 [0 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 2 2]
 [0 0 1 0 0 0 0 1 2]
 [1 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]]
[[0 0 0 0 0 0 2 1 0]
 [0 2 0 0 0 0 0 0 1]
 [0 0 0 

[[0 0 0 2 2 0 1 0 0]
 [0 0 0 0 0 0 0 0 0]
 [1 1 2 0 0 0 0 0 1]
 [0 1 1 0 0 0 0 0 2]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 2 0 0 0 0]
 [0 0 0 0 0 0 0 2 0]
 [0 1 0 0 2 0 0 0 0]
 [0 1 0 2 0 0 0 0 0]]
[[0 0 0 2 2 0 1 0 0]
 [1 0 0 0 0 0 0 0 0]
 [1 1 2 0 0 0 0 0 1]
 [0 1 1 0 0 0 0 0 2]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 2 0 0 0 0]
 [0 0 0 0 0 0 0 2 0]
 [0 1 0 0 2 0 0 0 0]
 [0 1 0 2 0 0 0 0 0]]
[[0 0 0 2 2 2 1 0 0]
 [1 0 0 0 0 0 0 0 0]
 [1 1 2 0 0 0 0 0 1]
 [0 1 1 0 0 0 0 0 2]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 2 0 0 0 0]
 [0 0 0 0 0 0 0 2 0]
 [0 1 0 0 2 0 0 0 0]
 [0 1 0 2 0 0 0 0 0]]
[[0 0 0 2 2 2 1 0 0]
 [1 0 0 0 0 0 0 0 0]
 [1 1 2 0 1 0 0 0 1]
 [0 1 1 0 0 0 0 0 2]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 2 0 0 0 0]
 [0 0 0 0 0 0 0 2 0]
 [0 1 0 0 2 0 0 0 0]
 [0 1 0 2 0 0 0 0 0]]
[[0 0 0 2 2 2 1 0 0]
 [1 0 0 0 0 0 0 0 0]
 [1 1 2 0 1 0 0 0 1]
 [0 1 1 0 0 0 0 0 2]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 2 0 2 0 0]
 [0 0 0 0 0 0 0 2 0]
 [0 1 0 0 2 0 0 0 0]
 [0 1 0 2 0 0 0 0 0]]
[[0 0 0 2 2 2 1 0 0]
 [1 0 0 0 0 0 0 0 0]
 [1 1 2 

[[2 0 1 2 2 2 1 1 2]
 [1 1 1 2 1 1 0 0 0]
 [1 1 2 1 1 1 0 2 1]
 [0 1 1 0 1 2 0 1 2]
 [1 2 0 1 2 2 1 2 0]
 [0 2 2 0 2 0 2 0 0]
 [1 2 1 2 0 1 1 2 1]
 [0 1 2 2 2 0 0 2 2]
 [1 1 1 2 0 0 2 2 2]]
[[2 0 1 2 2 2 1 1 2]
 [1 1 1 2 1 1 0 0 0]
 [1 1 2 1 1 1 0 2 1]
 [0 1 1 0 1 2 0 1 2]
 [1 2 0 1 2 2 1 2 0]
 [0 2 2 0 2 0 2 0 0]
 [1 2 1 2 1 1 1 2 1]
 [0 1 2 2 2 0 0 2 2]
 [1 1 1 2 0 0 2 2 2]]
[[2 0 1 2 2 2 1 1 2]
 [1 1 1 2 1 1 0 0 0]
 [1 1 2 1 1 1 0 2 1]
 [0 1 1 0 1 2 0 1 2]
 [1 2 0 1 2 2 1 2 0]
 [0 2 2 0 2 0 2 2 0]
 [1 2 1 2 1 1 1 2 1]
 [0 1 2 2 2 0 0 2 2]
 [1 1 1 2 0 0 2 2 2]]
cnt :  62
[[2 0 1 2 2 2 1 1 2]
 [1 1 1 2 1 1 0 0 0]
 [1 1 2 1 1 1 0 2 1]
 [0 1 1 0 1 2 0 1 2]
 [1 2 0 1 2 2 1 2 0]
 [0 2 2 0 2 0 2 2 0]
 [1 2 1 2 1 1 1 2 1]
 [0 1 2 2 2 0 0 2 2]
 [1 1 1 2 0 0 2 2 2]]
[[0 0 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]]
[[0 0 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0

KeyboardInterrupt: 